In [2]:
import pandas as pd
import os
#import google.generativeai as google

file_path = "data_clean.csv"
if os.path.exists(file_path):
    os.remove(file_path)


df = pd.read_csv("filtered_tweets_engie.csv", sep=";", encoding="utf-8")
df = df.dropna()
df = df.drop_duplicates()


df = df.drop(['id', 'screen_name', 'name'], axis=1)

df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df['created_at'] = df['created_at'].dt.tz_convert('Europe/Paris')
df['created_at'] = df['created_at'].dt.strftime('%d/%m/%Y %Hh')

#df.to_csv("data_clean.csv", index=False, encoding="ISO-8859-1", sep=";")
df.to_json("data_clean.json", orient="records", lines=True, force_ascii=False)

In [ ]:
%pip install mistralai
import time
import pandas as pd
import re
import string
import json
import os
from mistralai import Mistral

api_key = "7AWeZFTcIqhdTArxPtAtENMQip622bPH"
client = Mistral(api_key=api_key)


agent_id = "ag:7c5f296e:20250310:untitled-agent:ef957914"

df = pd.read_csv("cleaned_filtered_tweets_engie.csv", sep=";")
df["created_at"] = pd.to_datetime(
    df["created_at"], errors="coerce", utc=True
) 

if "full_text" not in df.columns:
    raise ValueError("Error: Column 'full_text' not found in CSV.")


def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"http\S+", "", text)
    text = text.translate(
        str.maketrans("", "", string.punctuation)
    ) 
    return text.strip()


df["clean_text"] = df["full_text"].apply(clean_text)


def classify_tweet_mistral(text):
    prompt = f"""
    You are an NLP agent for analyzing tweets.
    Return **ONLY a valid JSON** with these attributes:

    {{
        "category": "Facturation / Gaz / Électricité / Contrat / Autre",
        "sentiment": "Positif / Neutre / Négatif",
        "resolu": "Oui / Non",
        "inconfort": A score from 0 to 100,
        "urgence": true / false
    }}

    Tweet: "{text}"

    **Return only valid JSON output, no extra text.**
    """

    try:
        response = client.agents.complete(
            agent_id=agent_id, messages=[{"role": "user", "content": prompt}]
        )
        json_text = response.choices[0].message.content.strip()

        json_text = json_text[json_text.find("{") : json_text.rfind("}") + 1]

        return json_text 
    except Exception:
        return "Erreur"

results = []

for i in range(len(df)):
    tweet_text = df["clean_text"].iloc[i]
    result = classify_tweet_mistral(tweet_text)

    try:
        json_data = json.loads(result)
        results.append(json_data)
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format for tweet {i + 1}.")
        results.append(
            {
                "category": "Erreur",
                "sentiment": "Erreur",
                "resolu": "Erreur",
                "inconfort": "Erreur",
                "urgence": "Erreur",
            }
        )

    print(f"🔹 Tweet {i + 1} classified successfully.")
    time.sleep(5)

df_results = pd.DataFrame(results)

df_final = df.head(len(df_results)).copy()
df_final = pd.concat(
    [df_final.reset_index(drop=True), df_results.reset_index(drop=True)], axis=1
)

df_final.to_csv("tweets_classified_kpi_mistral.csv", sep=";", index=False)

print("✅ Analysis complete! Results saved to 'tweets_classified_kpi_mistral.csv'.")

Note: you may need to restart the kernel to use updated packages.
🔹 Tweet 1 classified successfully.
🔹 Tweet 2 classified successfully.
🔹 Tweet 3 classified successfully.


KeyboardInterrupt: 